In [6]:
import pandas as pd
import numpy as np
import os
import requests
from datetime import date
import random
from faker import Faker

fake = Faker()
fake_kor = Faker("ko_KR")

csv_path = r"C:\app\ojbs0\product\21c\oradata\entity"

In [16]:
# 1) Plans

plans_data = [
    {
        "plan_id": "F",
        "plan_type": "Free",
        "price": 0,
        "max_resolution": "480p",
        "max_concurrent_streams": 1,
        "downloadable": "N"
    },
    {
        "plan_id": "B",
        "plan_type": "Basic",
        "price": 8,
        "max_resolution": "1080p",
        "max_concurrent_streams": 2,
        "downloadable": "Y"
    },
    {
        "plan_id": "P",
        "plan_type": "Premium",
        "price": 15,
        "max_resolution": "4K",
        "max_concurrent_streams": 4,
        "downloadable": "Y"
    }
]

df_plans = pd.DataFrame(plans_data)
file_name = "plans.csv"
save_path = os.path.join(csv_path, file_name)
df_plans.to_csv(save_path, index=False)

In [17]:
# 2) Movies
# 영화진흥위원회 오픈API에서 실제 영화 정보 일부 추출

API_KEY = "d5209c49ea7de1971faa246cdd4f87b5"    # 사용자 키

movies_data = []

for page in range(1, 300):
    params = {
        "key": API_KEY,
        "itemPerPage": 100,
        "curPage": page,
        "openStartDt": 1980,
        "openEndDt": 2024
    }

    try:
        res = requests.get(
            "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json",
            params=params
        ).json()

        # 정상 응답인지 확인
        if "movieListResult" in res and "movieList" in res["movieListResult"]:
            movies_data.extend(res["movieListResult"]["movieList"])
        else:
            print(f"page {page} 응답에 movieListResult 없음: {res}")

    except Exception as e:
        print(f"page {page} 요청 실패: {e}")

# 필요한 영화 개수만큼 랜덤 id 생성 (중복 없음)
unique_ids = random.sample(range(10000, 100000), k=len(movies_data))

processed_movies = []
for idx, m in enumerate(movies_data):
    open_dt = m.get("openDt", "")
    genre = m.get("repGenreNm")
    showtime = random.randint(80, 180)  # 다른 url까지 갖고오면 너무 무거워져서 임의로 상영시간 설정

    # free 유저가 관람 가능한 영화 여부
    is_not_free = "Y" if (open_dt and int(open_dt[:4]) >= 2023) else "N"

    # 장르가 불분명하거나 성인 영화는 제외
    if not genre or "성인물(에로)" in genre or "멜로/로맨스" in genre:
        continue

    processed_movies.append({
        "movie_id": unique_ids[idx],
        "title": m.get("movieNm"),
        "genre": genre,
        "open_dt": open_dt,
        "showtime": showtime,
        "is_not_free": is_not_free
    })

df_movies = pd.DataFrame(processed_movies)

# 중복 제거 (제목+개봉일 기준)
df_movies.drop_duplicates(subset=["title", "open_dt"], inplace=True)

# 영화 10000편 추출
df_movies = df_movies.sample(n=min(10000, len(df_movies)), random_state=42)

file_name = "movies.csv"
save_path = os.path.join(csv_path, file_name)
df_movies.to_csv(save_path, index=False)

In [13]:
# 3) Members

N_MEMBERS = 600000

unique_ids = random.sample(range(1000000, 10000000), k=N_MEMBERS)

members = []
for i in range(1, N_MEMBERS + 1):

    plans = ["Free", "Basic", "Premium"]
    plan_probs = np.random.dirichlet([0.4, 1.4, 1.36])
    plan = random.choices(plans, weights=plan_probs, k=1)[0]

    age_groups = [
        (14, 19),
        (20, 29),
        (30, 39),
        (40, 49),
        (50, 59),
        (60, 69),
        (70, 79)
    ]
    age_probs = np.random.dirichlet([1, 2, 1.8, 1.6, 1.2, 0.8, 0.2])
    chosen_group = random.choices(age_groups, weights=age_probs, k=1)[0]
    age = random.randint(chosen_group[0], chosen_group[1])

    members.append({
        "member_id": unique_ids[i-1],
        "name": fake_kor.name(),
        "email": fake.email(),
        "age": age,
        "join_date": fake.date_between(start_date=date(2024,1,1), end_date=date(2024,12,25)),
        "current_plan": plan
    })

df_members = pd.DataFrame(members)
file_name = "members.csv"
save_path = os.path.join(csv_path, file_name)
df_members.to_csv(save_path, index=False)

In [14]:
# 4) Subscription

subscriptions = []
subscription_id = 1

periods = pd.date_range("2024-01-01", "2024-12-31", freq="MS").strftime("%Y-%m").tolist()

plans = ["F", "B", "P"]

for _, row in df_members.iterrows():
    member_id = row["member_id"]
    join_month = row["join_date"].strftime("%Y-%m")

    for p in periods:
        if p < join_month:  # 가입 이전 달은 제외
            continue

        plan_probs = np.random.dirichlet([1.1, 1.9, 1.95])
        plan_id = random.choices(plans, weights=plan_probs, k=1)[0]

        subscriptions.append({
            "subscription_id": subscription_id,
            "member_id": member_id,
            "period": p,
            "plan_id": plan_id,
            "is_new": "Y" if p == join_month else "N"
        })

        subscription_id += 1

df_subs = pd.DataFrame(subscriptions)
file_name = "subscription.csv"
save_path = os.path.join(csv_path, file_name)
df_subs.to_csv(save_path, index=False)

In [ ]:
# 5) Viewing History

viewing_history = []
view_id = 0

df_movies = pd.read_csv(os.path.join(csv_path, "movies.csv"))
df_movies["open_dt"] = pd.to_datetime(df_movies["open_dt"], errors="coerce").dt.date
df_movies = df_movies.dropna(subset=["open_dt"])

movie_ids = df_movies["movie_id"].tolist()
movie_opens = dict(zip(df_movies["movie_id"], df_movies["open_dt"]))

statuses = ["Started", "Partial", "Completed"]
devices = ["Mobile", "Tablet", "PC", "TV"]

for _, row in df_members.iterrows():
    member_id = row["member_id"]
    join_date = pd.to_datetime(row["join_date"]).date()

    n_views = max(0, int(random.gauss(8, 7)))

    for _ in range(n_views):
        watch_date = fake.date_between(start_date=join_date, end_date=date(2024, 12, 31))

        movie_id = random.choice(movie_ids)
        movie_open = movie_opens[movie_id]

        # 시청일이 개봉일보다 빠르면 스킵
        if watch_date < movie_open:
            continue

        view_id += 1

        status_probs = np.random.dirichlet([1, 2, 7])
        status = random.choices(statuses, weights=status_probs, k=1)[0]

        device_probs = np.random.dirichlet([3, 2.6, 3.2, 1.2])
        device = random.choices(devices, weights=device_probs, k=1)[0]

        viewing_history.append({
            "view_id": view_id,
            "member_id": member_id,
            "movie_id": movie_id,
            "watch_date": watch_date,
            "watch_status": status,
            "main_device_type": device
        })

df_viewing = pd.DataFrame(viewing_history)
file_name = "viewing_history.csv"
save_path = os.path.join(csv_path, file_name)
df_viewing.to_csv(save_path, index=False)

KeyboardInterrupt: 